In [123]:
import numpy as np
from pydrake.all import (
    RigidTransform,
    RotationMatrix,
    StartMeshcat,
    RandomGenerator,
    Diagram,
)
import numpy as np

import sponana.utils
from sponana.planner.rrt import SpotProblem, rrt_planning
from sponana.planner.navigator import Navigator
from sponana.fsm.finite_state_machine import FiniteStateMachine
import sponana.sim

In [124]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7011


In [125]:
"""
meshcat.Delete()
meshcat.DeleteAddedControls()
rng = np.random.default_rng(145)  # this is for python
generator = RandomGenerator(rng.integers(0, 1000))  # this is for c++

add_spot = True
# simulation_time = -1  # run indefinitely until ESC is pressed
simulation_time = 2
debug = True
add_fixed_cameras = False
enable_arm_ik = True  # turn this off if you find the arm too annoying
use_teleop = False

simulator, diagram = sponana.sim.clutter_gen(
    meshcat,
    rng,
    debug=debug,
    simulation_time=simulation_time,
    add_spot=add_spot,
    add_fixed_cameras=add_fixed_cameras,
    enable_arm_ik=enable_arm_ik,
    use_teleop=use_teleop,
)
"""

'\nmeshcat.Delete()\nmeshcat.DeleteAddedControls()\nrng = np.random.default_rng(145)  # this is for python\ngenerator = RandomGenerator(rng.integers(0, 1000))  # this is for c++\n\nadd_spot = True\n# simulation_time = -1  # run indefinitely until ESC is pressed\nsimulation_time = 2\ndebug = True\nadd_fixed_cameras = False\nenable_arm_ik = True  # turn this off if you find the arm too annoying\nuse_teleop = False\n\nsimulator, diagram = sponana.sim.clutter_gen(\n    meshcat,\n    rng,\n    debug=debug,\n    simulation_time=simulation_time,\n    add_spot=add_spot,\n    add_fixed_cameras=add_fixed_cameras,\n    enable_arm_ik=enable_arm_ik,\n    use_teleop=use_teleop,\n)\n'

In [126]:
def update_do_rrt(current_cam_reached, current_cam_ind, check_banana):
        """
        Function to update flag to indicate to RRT/Navigator for ready for planning/movement.
        Inputs: 
        - from context, 
        current_cam_reached: int where 0 when camera is reached and 1 when not.
        
        Returns: 
        - new_cam_reached: if current cam reached, switch to 0 so that navigator can plan again. 
        if current cam is not reached, stay 1 so that navigator will wait. 
        """
        do_rrt = 0
        #just starting, have not reached the first camera pose, do_rrt to get to the first camera
        if current_cam_reached == 0 and current_cam_ind == 0:
            print("first if case")
            do_rrt = 1
        #have reached camera, and banana has finished being checked
        elif current_cam_reached == 1 and check_banana == 1: 
            print("second if case")
            do_rrt = 1
        return do_rrt

In [127]:
def update_camera_ind(current_cam_reached, see_banana, has_banana, current_cam_ind):
        """Function for updating camera pose list index. 
        Inputs: 
        - from context: 
        current_cam_reached: int where 0 when camera is reached and 1 when not.
        see_banana: int where 0 when banana is not seen and 1 when banana seen. 
        has_banana: int where 0 when banana is grasped nad 1 when banana is not grasped. 

        Returns: None
        If current camera is reached and no banana is seen/grasped, needs to continue to search
        next camera pose, so current_cam_ind is incremented. 
        """ 
        if current_cam_reached == 1 and see_banana == 0 and has_banana == 0:
            if current_cam_ind <= 2:
                current_cam_ind += 1
            #none viewpoints have bananas, so do it all again?
            else:
                current_cam_ind = 0

        return current_cam_ind


In [128]:
def get_camera_pose(current_cam_ind, camera_pose_list):
        """
        Function to get the next camera pose for Spot to travel to in RRT
        Inputs: 
        - from context: 
        current_cam_reached: int where 0 when camera is reached and 1 when not.
        Returns: 
        - next camera pose
        """
        next_camera_pose = camera_pose_list[current_cam_ind]
        return next_camera_pose

In [129]:
def update_check_banana(current_cam_reached, see_banana, has_banana):
        """Function as indicater for perception module.
        Inputs: 
        - from context: 
        current_cam_reached: int where 0 when camera is reached and 1 when not.
        see_banana: int where 0 when banana is not seen and 1 when banana seen. 
        has_banana: int where 0 when banana is grasped nad 1 when banana is not grasped. 

        Returns: 
        check_banana: if current_cam is reached, and banana is not seen, and banana is not grasped, 
        return 1 to call the perception module/system. 
        """
        check_banana = 0
        if current_cam_reached == 1 and see_banana == 0 and has_banana == 0:
            check_banana = 1
        return check_banana

In [130]:
def update_grasp_banana(current_cam_reached, see_banana, has_banana):
        """Function as indicater for perception module.
        Inputs: 
        - from context: 
        current_cam_reached: int where 0 when camera is reached and 1 when not.
        see_banana: int where 0 when banana is not seen and 1 when banana seen. 
        has_banana: int where 0 when banana is grasped nad 1 when banana is not grasped. 

        Returns: 
        grasp_banana: if current_cam is reached, and banana is seen, and banana is not grasped, 
        return 1 to get banana grasped system. 
        """
        grasp_banana = 0
        if current_cam_reached == 1 and see_banana == 1 and has_banana == 0:
            grasp_banana = 1
        return grasp_banana

In [131]:
def update_completion(has_banana):
    completed = 0
    if has_banana == 1:
        completed = 1
    return completed

In [132]:
def dummy_call_perception(check_banana, current_cam_ind):
    see_banana = 0
    if check_banana == 1:
        if current_cam_ind == 2: #let's just pretend this is the one with banana
            see_banana = 1
            print("found banana")
        else:
            print("banana not found")
    return see_banana
    


In [133]:
def dummy_call_grasp_banana(grasp_banana):
    has_banana = 0
    if grasp_banana == 1:
        has_banana = 1
    return has_banana

In [134]:
# visualize RRT output
from manipulation.meshcat_utils import AddMeshcatTriad


def visualize_path(path):
    for i, pose in enumerate(path):
        pose = RigidTransform(RotationMatrix.MakeZRotation(pose[2]), [*pose[:2], 0.0])
        opacity = 0.2
        AddMeshcatTriad(meshcat, f"trajectory_{i}", X_PT=pose, opacity=opacity)

In [135]:
initial_pose = np.array([1.00000000e00, 1.50392176e-12, 3.15001955e00])
q_start = initial_pose

camera_pose0 = np.array([1, -2.5, 0.2475])
camera_pose1 = np.array([1.00000000e00, 1.50392176e-12, 3.15001955e00])
camera_pose2 = np.array([-2, -2, 0.2475])
camera_pose_list = [camera_pose0, camera_pose1, camera_pose2]

In [136]:
def dummy_rrt(q_goal):
    print("q_sub_goal:", q_goal, "rrt done")
    current_cam_reached = 1
    return current_cam_reached


In [137]:
def check_do_rrt_unit_test():
    current_cam_reached = 0 
    current_cam_ind = 0
    check_banana = 0
    check_do_rrt = update_do_rrt(current_cam_reached, current_cam_ind, check_banana)
    if check_do_rrt == 1:
        print("Camera not reached, ind = 0, need to do rrt, correct")
    current_cam_reached = 1 
    current_cam_ind = 0
    check_banana = 0


In [138]:

"""navigator = Navigator()

def execute_finite_state_machine():
    current_cam_reached = 0 
    current_cam_ind = 0
    check_banana = 0
    see_banana = 0
    has_banana = 0 
    grasp_banana = 0
    completed = 0 #update_completion(has_banana)
    loop_count = 0
    
    while completed == 0:
        print("loop_count", loop_count, "_____")
        check_do_rrt = update_do_rrt(current_cam_reached, current_cam_ind, check_banana)
        print("check do rrt", check_do_rrt)
        if check_do_rrt == 1:
            #Do RRT
            print("get pose")
            q_sub_goal = get_camera_pose(current_cam_ind, camera_pose_list)
            #spot_problem = SpotProblem(q_start=q_start, q_goal=q_sub_goal, collision_checker=navigator._collision_check)
            #path = rrt_planning(spot_problem, 1000, 0.05)
            #print("q_sub_goal:", q_sub_goal, "rrt done")
            current_cam_reached = dummy_rrt(q_sub_goal)
        
        see_banana = dummy_call_perception(check_banana, current_cam_ind)
        print("see_banana", see_banana)
        has_banana = dummy_call_grasp_banana(grasp_banana)
        print("has_banana", has_banana)
        current_cam_ind = update_camera_ind(current_cam_reached, see_banana, has_banana, current_cam_ind)
        print("current_cam_ind", current_cam_ind)
        check_banana = update_check_banana(current_cam_reached, see_banana, has_banana)
        print("check_banana", check_banana)
        grasp_banana = update_grasp_banana(current_cam_reached, see_banana, has_banana)
        print("grasp_banana",grasp_banana)
        completed = update_completion(has_banana)
        print("completed", completed)
        current_cam_reached = 0
        loop_count += 1
"""


'navigator = Navigator()\n\ndef execute_finite_state_machine():\n    current_cam_reached = 0 \n    current_cam_ind = 0\n    check_banana = 0\n    see_banana = 0\n    has_banana = 0 \n    grasp_banana = 0\n    completed = 0 #update_completion(has_banana)\n    loop_count = 0\n    \n    while completed == 0:\n        print("loop_count", loop_count, "_____")\n        check_do_rrt = update_do_rrt(current_cam_reached, current_cam_ind, check_banana)\n        print("check do rrt", check_do_rrt)\n        if check_do_rrt == 1:\n            #Do RRT\n            print("get pose")\n            q_sub_goal = get_camera_pose(current_cam_ind, camera_pose_list)\n            #spot_problem = SpotProblem(q_start=q_start, q_goal=q_sub_goal, collision_checker=navigator._collision_check)\n            #path = rrt_planning(spot_problem, 1000, 0.05)\n            #print("q_sub_goal:", q_sub_goal, "rrt done")\n            current_cam_reached = dummy_rrt(q_sub_goal)\n        \n        see_banana = dummy_call_percep

In [141]:
navigator = Navigator()

def execute_finite_state_machine():
    current_cam_reached = 0 
    current_cam_ind = 0
    check_banana = 0
    see_banana = 0
    has_banana = 0 
    grasp_banana = 0
    completed = 0 #update_completion(has_banana)
    loop_count = 0
    
    check_do_rrt = update_do_rrt(current_cam_reached, current_cam_ind, check_banana)
    print("check do rrt", check_do_rrt)
    if check_do_rrt == 1:
        #Do RRT
        print("get pose")
        q_sub_goal = get_camera_pose(current_cam_ind, camera_pose_list)
            #spot_problem = SpotProblem(q_start=q_start, q_goal=q_sub_goal, collision_checker=navigator._collision_check)
            #path = rrt_planning(spot_problem, 1000, 0.05)
            #print("q_sub_goal:", q_sub_goal, "rrt done")
        current_cam_reached = dummy_rrt(q_sub_goal)
    check_banana = update_check_banana(current_cam_reached, see_banana, has_banana)
    print("check_banana", check_banana)
    see_banana = dummy_call_perception(check_banana, current_cam_ind)
    print("see_banana", see_banana)
    grasp_banana = update_grasp_banana(current_cam_reached, see_banana, has_banana)
    print("grasp_banana",grasp_banana)
    has_banana = dummy_call_grasp_banana(grasp_banana)
    print("has_banana", has_banana)
    current_cam_ind = update_camera_ind(current_cam_reached, see_banana, has_banana, current_cam_ind)
    print("current_cam_ind", current_cam_ind)
    completed = update_completion(has_banana)
    print("completed", completed)
    current_cam_reached = 0
    loop_count += 1

In [142]:
execute_finite_state_machine()

first if case
check do rrt 1
get pose
q_sub_goal: [ 1.     -2.5     0.2475] rrt done
check_banana 1
grasp_banana 0
banana not found
see_banana 0
has_banana 0
current_cam_ind 1
completed 0
